In [ ]:
import pandas as pd
import numpy as np

Data = pd.read_csv(r"E:\MIT-6\ML\ML Algos Lab\Data\PlayTennis.csv")

# Define a function to split the data based on a continuous attribute and a split value
def split_continuous_data(df, feature_name, split_value):
    left_df = df[df[feature_name] <= split_value]
    right_df = df[df[feature_name] > split_value]
    return left_df, right_df

# Calculate the entropy for the whole dataset (Based on no of yes's and no's)
def totalEntropy(df, labelname, labellist):
    rows = df.shape[0]
    entropy = 0
    
    for cname in labellist:
        cnt = df[df[labelname] == cname].shape[0]
        classEntropy = -(cnt/rows)*np.log2(cnt/rows)
        entropy += classEntropy
        
    return entropy

# Calculate the entropy for a specific feature
def specificEntropy(fval, labelname, labellist):
    cnt = fval.shape[0]
    entropy = 0
    
    for cname in labellist : 
        rows = fval[fval[labelname] == cname].shape[0]
        classentropy = 0
        if rows != 0:
            classProb = rows/cnt
            classentropy = -classProb * np.log2(classProb)
        entropy += classentropy
        
    return entropy

# Calculate the split info gain ratio for each feature
def splitInfoGainRatio(fname, df,labelname, labellist):
    fvals = sorted(df[fname].unique()) # sort the unique values of the attribute in increasing order
    rows = df.shape[0]
    finfo = 0.0
    splitinfo = 0.0
    splitpoints = [(fvals[i] + fvals[i+1]) / 2 for i in range(len(fvals) - 1)] # calculate the midpoint values as potential split points
    
    for sp in splitpoints:
        fdata1 = df[df[fname] < sp] # rows with values less than split point
        fdata2 = df[df[fname] >= sp] # rows with values greater than or equal to split point
        fvalCnt1 = fdata1.shape[0]
        fvalCnt2 = fdata2.shape[0]
        fvalProb1 = fvalCnt1 / rows
        fvalProb2 = fvalCnt2 / rows
        finfo += (fvalProb1*specificEntropy(fdata1, labelname, labellist) + fvalProb2*specificEntropy(fdata2, labelname, labellist))
        splitinfo -= (fvalProb1*np.log2(fvalProb1) + fvalProb2*np.log2(fvalProb2))
    
    #if split=0, gain ratio will lead to div by 0 so instead return 0
    if splitinfo == 0:
        return 0
    else:
        return (totalEntropy(df,labelname, labellist) - finfo) / splitinfo